In [20]:
import pandas as pd
import itertools
import copy

In [21]:
def gen_missing_comb_multi(df):
    new_rows = []
    columns = df.columns.tolist()
    for col_num in range(1, len(columns) + 1):
        for col_comb in itertools.combinations(columns, col_num):
            for i, row in df.iterrows():
                new_row = copy.deepcopy(row)
                for col in col_comb:
                    new_row[col] = "UNKNOWN"
                new_rows.append(new_row)
    return pd.DataFrame(new_rows, columns=df.columns)

In [22]:
# List of data
df = pd.read_excel(r"C:\Users\Axel\Documents\PYTHON\FragHub\FragHub\TOOLS\generate_decision_tree\Instrument_tree.xlsx", sheet_name="Agilent")
df = df.astype(str)

In [23]:
# Generate the DataFrame with missing combinations
df_missing_comb_multi = gen_missing_comb_multi(df)

In [24]:
print(df_missing_comb_multi)

     MARQUES   MODELS SPECTRUM_TYPE INSTRUMENT_TYPE IONISATION RESOLUTION
0    UNKNOWN     6210            CE             TOF        ESI       HIGH
1    UNKNOWN     6410            LC             QQQ        ESI        LOW
2    UNKNOWN     6530            LC            QTOF        ESI       HIGH
3    UNKNOWN     6531            LC            QTOF        ESI       HIGH
4    UNKNOWN     6532            LC            QTOF        ESI       HIGH
..       ...      ...           ...             ...        ...        ...
150  UNKNOWN  UNKNOWN       UNKNOWN         UNKNOWN    UNKNOWN    UNKNOWN
151  UNKNOWN  UNKNOWN       UNKNOWN         UNKNOWN    UNKNOWN    UNKNOWN
152  UNKNOWN  UNKNOWN       UNKNOWN         UNKNOWN    UNKNOWN    UNKNOWN
153  UNKNOWN  UNKNOWN       UNKNOWN         UNKNOWN    UNKNOWN    UNKNOWN
154  UNKNOWN  UNKNOWN       UNKNOWN         UNKNOWN    UNKNOWN    UNKNOWN

[9765 rows x 6 columns]


In [25]:
df_missing_comb_multi = pd.DataFrame(df_missing_comb_multi, columns=["MARQUES", "MODELS", "SPECTRUM_TYPE", "INSTRUMENT_TYPE", "IONISATION", "RESOLUTION"])

In [26]:
def count_non_unknown(row):
    return sum(row != 'UNKNOWN')

# Apply this function to each row
non_unknown_counts = df_missing_comb_multi.apply(count_non_unknown, axis=1)

# Filter out rows where only 'MODELS' or no columns are not 'UNKNOWN'
df_missing_comb_multi = df_missing_comb_multi[(non_unknown_counts > 1) | ((non_unknown_counts == 1) & (df_missing_comb_multi['MODELS'] != 'UNKNOWN'))]

In [27]:
def should_exclude_row_2(row):
    non_unknown_inds = [index for index, value in row.items() if value != 'UNKNOWN']
    if len(non_unknown_inds) == 2 and set(non_unknown_inds) != {'MARQUES', 'MODELS'}:
        return True
    return False

rows_to_exclude = df_missing_comb_multi.apply(should_exclude_row_2, axis=1)
df_filtered = df_missing_comb_multi.loc[~rows_to_exclude]

In [28]:
def should_exclude_row_resolution(row):
    if row['RESOLUTION'] == 'UNKNOWN':
        return True
    return False

rows_to_exclude_resolution = df_filtered.apply(should_exclude_row_resolution, axis=1)
df_filtered = df_filtered.loc[~rows_to_exclude_resolution]

In [29]:
def should_exclude_row_3(row):
    non_unknown_inds = [index for index, value in row.items() if value != 'UNKNOWN']
    if len(non_unknown_inds) == 2 and set(non_unknown_inds) != {'MARQUES', 'SPECTRUM_TYPE'}:
        return True
    return False

rows_to_exclude = df_filtered.apply(should_exclude_row_3, axis=1)
df_filtered = df_filtered.loc[~rows_to_exclude]

In [30]:
def should_exclude_row_4(row):
    non_unknown_inds = [index for index, value in row.items() if value != 'UNKNOWN']
    if len(non_unknown_inds) == 2 and set(non_unknown_inds) != {'MARQUES', 'INSTRUMENT_TYPE'}:
        return True
    return False

rows_to_exclude = df_filtered.apply(should_exclude_row_4, axis=1)
df_filtered = df_filtered.loc[~rows_to_exclude]

In [31]:
# def should_exclude_row_5(row):
#     non_unknown_inds = [index for index, value in row.items() if value != 'UNKNOWN']
#     if len(non_unknown_inds) == 2 and set(non_unknown_inds) != {'MARQUES', 'IONISATION'}:
#         return True
#     return False
# 
# rows_to_exclude = df_filtered.apply(should_exclude_row_5, axis=1)
# df_filtered = df_filtered.loc[~rows_to_exclude]

In [32]:
def should_exclude_row_6(row):
    non_unknown_inds = [index for index, value in row.items() if value != 'UNKNOWN']
    if len(non_unknown_inds) == 2 and set(non_unknown_inds) != {'MARQUES', 'RESOLUTION'}:
        return True
    return False

rows_to_exclude = df_filtered.apply(should_exclude_row_6, axis=1)
df_filtered = df_filtered.loc[~rows_to_exclude]

In [33]:
def should_exclude_row_7(row):   
    if row['MODELS'] == 'UNKNOWN' and row['INSTRUMENT_TYPE'] == 'UNKNOWN':
        return True
    return False

rows_to_exclude = df_filtered.apply(should_exclude_row_7, axis=1)
df_filtered = df_filtered.loc[~rows_to_exclude]

In [34]:
def should_exclude_row_8(row):
    if row['MODELS'] == 'UNKNOWN' and row['IONISATION'] == 'UNKNOWN':
        return True
    return False

rows_to_exclude = df_filtered.apply(should_exclude_row_8, axis=1)
df_filtered = df_filtered.loc[~rows_to_exclude]

In [35]:
# Write to CSV
df_filtered.to_excel(r"C:\Users\Axel\Documents\PYTHON\FragHub\FragHub\TOOLS\generate_decision_tree\permutations.xlsx", index=False)